# Criação de Dashboard financeiro com dados pessoais
## Objetivo
Extrair os dados de gastos pessoais da planilha de controle de despesas que é utilizada desde 2014 para demonstrar técnicas de tratamento de dados não estruturados.
## Procedimento
O projeto irá seguir os seguintes passos:
* Acessar o material através da API do Google Sheets, onde a planilha se encontra
* Categorizar os dados por mês de ocorrência e categoria
* Realizar análise das despesas ao longo do tempo
* Disponibilizar os dados para consumo pelo PowerBI para criação de um Dashboard acessível online
* Produzir dados anonimizados para divulgação online e criação de Portfolio
#Execução
## Acessando as planilhas do Google Sheets
O acesso direto ao Google Sheets se faz necessário pois as planilhas ainda são utilizadas e atualizadas semanalmente, portanto, o script deverá ser rodado periodicamente para manter o Dashboard atualizado.

In [ ]:
# importação das bibliotecas necessárias
import gspread
from oauth2client.service_account import ServiceAccountCredentials
import pandas as pd
import os
import numpy as np

#Configuração da Autenticação
scope = ["https://spreadsheets.google.com/feeds", 'https://www.googleapis.com/auth/drive']

'''
Para um script completo, utilizar a linha abaixo para obter o caminho do
arquivo de credenciais de forma dinâmica,
garantindo que funcione independentemente do
local onde o script seja executado.

> script_dir = os.path.dirname(os.path.abspath(__file__))

Para o caso do notebook iremos utilizar o cwd para obter
o caminho do arquivo de credenciais.
'''

script_dir = os.getcwd()
creds_path = os.path.join(script_dir, 'credentials.json')
creds = ServiceAccountCredentials.from_json_keyfile_name(creds_path, scope)
client = gspread.authorize(creds)
planilha = client.open('Orçamento - João')

## Importando as abas da planilha
Cada planilha da tabela possui uma formatação diferente ao longo dos anos, com um formato mais estável a partir de 2016 então teremos uma metodologia diferente por período.
### 2014
A planilha apresenta na sua primeira coluna a descrição e nas demais os meses do ano, sendo que os valores foram sendo inseridos a partir do topo em cada virada de mês, resultando em uma planilha com muitos vazios por mês.
Também não foi utilizada uma categorização padronizada de forma que a análise do comportamento de consumo se torna impraticável.
A estratégia de tratamento dos dados deste ano será:
* Dividir a tabela em subconjuntos por mês copiando a coluna de descrição e valor daquele mês
* Inserir a coluna de datas de cada subconjunto
* Empilhar as subseções para criar um único df com Descrição, Data e valor
* Limpar valores nulos decorrentes do método de divisão e agrupamento dos subconjuntos
* Categorizar os valores por receita e despesa de acordo com o sinal positivo ou negativo dos valores inseridos

In [ ]:
# Acessar a aba de 2014
plan2014 = planilha.worksheet('2014')

# Obter os cabeçalhos da planilha de 2014
headers2014 = plan2014.row_values(1)

# Filtrar os cabeçalhos para remover os vazios
headers2014 = [h for h in headers2014 if h.strip()]

# importar os dados da planilha de 2014 somente dos cabeçalhos válidos
data2014 = plan2014.get_all_records(expected_headers=headers2014)
df2014 = pd.DataFrame(data2014).iloc[:227,:]
df2014

,Despesas 2014,Categoria,Parcela,fevereiro,março,abril,maio,junho,julho,agosto,setembro,outubro,novembro,dezembro,
0,,,,,,,,,,,,,,,
1,Aluguel,Essenciais,1,,-925,,-950,-950,-910,-950,-1200,-1200,-1200,-1200,
2,Condomínio,Essenciais,1,,-289,-28355,-285,-285,-285,-285,-285,-23355,-285,,-685
3,Tanomoshi,Dívidas e Financiamentos,1,,3700,,3465,,,,,,,,
4,Mobilia,Essenciais,1,,-1262,,1300,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
222,Empréstimo errado,Dívidas e Financiamentos,1,,,,,,,,,,,-2,
223,,,,,,,,,,,,,,,
224,Entrada X,Receitas,1,,,,,,,,,,,2400,
225,,,,,,,,,,,,,,,


Com o dataframe de 2014 criado, podemos prossegir para a separação das colunas de cada mês e parear com as descrições.

In [ ]:
# Seleção apenas das colunas de interesse para criação do subconjunto
df2014_fev = df2014[['Despesas 2014','fevereiro']]
df2014_fev

,Despesas 2014,fevereiro
0,,
1,Aluguel,
2,Condomínio,
3,Tanomoshi,
4,Mobilia,
...,...,...
222,Empréstimo errado,
223,,
224,Entrada X,
225,,


Para um prcesso mais automatizado podemos criar um loop que gera as combinações de descrição e valor, além de adicionar uma coluna de data para o subset.

In [ ]:
# Criamos uma lista vazia que irá abrigar os subconjuntos criados no loop
subsets_2014 = []

# Criamos também um dicionário que relaciona as labels de meses do cabeçalho
# com as datas que queremos inserir na coluna de data
map_datas2014 = {
    'janeiro': '01/01/2014',
    'fevereiro': '01/02/2014',
    'março': '01/03/2014',
    'abril': '01/04/2014',
    'maio': '01/05/2014',
    'junho': '01/06/2014',
    'julho': '01/07/2014',
    'agosto': '01/08/2014',
    'setembro': '01/09/2014',
    'outubro': '01/10/2014',
    'novembro': '01/11/2014',
    'dezembro': '01/12/2014'
}

# Abrimos o loop definindo as variáveis e iremos iterar sobre os itens do dict
for mes , data_despesa in map_datas2014.items():
    # Inicialmente verificamos se existe o mês do dict dentro do df de 2014
    if mes not in df2014.columns:
        continue

    # Criamos o primeiro subset selecionando a coluna de descrição e a
    # coluna do respectivo mês
    subset_df = df2014[['Despesas 2014', mes]].copy()

    # Adicionamos a coluna de data no formato definido no dict
    subset_df['Data'] = data_despesa

    # Renomeamos as colunas para o padrão desejado
    subset_df.rename(columns={mes: 'Valor','Despesas 2014':'Descrição'}, inplace=True)

    # Inserimos o subset dentro do contâiner criado anteriormente
    subsets_2014.append(subset_df)

# Transformamos a lista de dataframes num único dataframe para realizar
# a limpeza dos dados
despesas_2014 = pd.concat(subsets_2014, ignore_index=True)

# Vamos trocar o símbolo de decimal brasileiro para o americano
# para consumo pela ferramenta de BI e deixamos a configuração de visualização
# para a ferramenta de análise

# Para a substituição primeiro convertemos os valores para string para que
# possamos utilizar o método .replace() e depois convertemos de volta para número
despesas_2014['Valor'] = pd.to_numeric(
    despesas_2014['Valor'].astype(str).str.replace(',', '.'),
    errors='coerce'
)

# Removemos linhas vazias da coluna valor
despesas_2014 = despesas_2014.dropna(subset=['Valor'])

# Removemos linhas onde o valor é 0
despesas_2014 = despesas_2014[despesas_2014['Valor'] != 0]

# Removemos as linhas onde temos descrição vazia
despesas_2014 = despesas_2014[despesas_2014['Descrição'] != '']

despesas_2014

,Descrição,Valor,Data
228,Aluguel,-925.0,01/03/2014
229,Condomínio,-289.0,01/03/2014
230,Tanomoshi,3700.0,01/03/2014
231,Mobilia,-1262.0,01/03/2014
232,Taiko,-60.0,01/03/2014
...,...,...,...
2489,Doce,-16.0,01/12/2014
2490,Encargos,-2.0,01/12/2014
2491,Remédios,-23.0,01/12/2014
2492,Empréstimo errado,-2.0,01/12/2014


Para verificar a integridade dos dados, vamos verificar se temos ocorrência de valores sem descrição, pois ela será necessária quando formos sumarizar os dados em categorias posteriormente.

In [ ]:
despesas_2014.loc[despesas_2014['Descrição'] == '']

,Descrição,Valor,Data


Por fim vamos adicionar a coluna de tipo para diferenciar os valores entre receita e despesa, além de transformar todos os valores em positivos.

In [ ]:
# Criamos a coluna tipo com base no sinal da coluna valor
despesas_2014['Tipo'] = np.where(despesas_2014['Valor'] < 0, 'D', 'R')

# Removemos então o sinal da coluna valor
despesas_2014['Valor'] = despesas_2014['Valor'].abs()

despesas_2014

,Descrição,Valor,Data,Tipo
228,Aluguel,925.0,01/03/2014,D
229,Condomínio,289.0,01/03/2014,D
230,Tanomoshi,3700.0,01/03/2014,R
231,Mobilia,1262.0,01/03/2014,D
232,Taiko,60.0,01/03/2014,D
...,...,...,...,...
2489,Doce,16.0,01/12/2014,D
2490,Encargos,2.0,01/12/2014,D
2491,Remédios,23.0,01/12/2014,D
2492,Empréstimo errado,2.0,01/12/2014,D


### 2015
Em 2015 temos uma disposição semelhante a 2014 então podemos reproduzir o mesmo processo.

In [ ]:
# Importamos a aba
plan2015 = planilha.worksheet('2015')

# Extraímos os cabeçalhos para passar ao método .get_all_records
headers2015 = plan2015.row_values(1)

# Criamos a lista de cabeçalhos excluido os vazios
headers2015 = [h for h in headers2015 if h.strip()]

# Extraímos os dados da planilha já filtrando as colunas
data2015 = plan2015.get_all_records(expected_headers=headers2015,numericise_ignore=['all'])

# Convertemos para um dataframe para manipulação
df2015 = pd.DataFrame(data2015)

# Vamos renomear a coluna na origem para simplificar o código mais adiante
df2015.rename(columns={'Despesas 2015': 'Descrição'}, inplace=True)

# Removemos as colunas 'Tipo', 'Categoria' e 'Parcela' do DataFrame de 2015
# pois não serão necessárias
df2015 = df2015.loc[:625, ~df2015.columns.isin(['Tipo', 'Categoria','Parcela'])]

df2015

,Descrição,Janeiro,Fevereiro,Março,Abril,Maio,Junho,Julho,Agosto,Setembro,Outubro,Novembro,Dezembro
0,Salário JV,2746,2624,2568,"2637,44",2703,2909,2900,1626,1901,2699,1559,1560
1,Adiantamento,,,,,,,,,,,1352,0
2,13º João,,,,,,,"833,41",,,,1000,1535
3,Saque VGBL,,,,,,,,,"3826,98",,,
4,WiiU extraviado,,,,,,,,,-1350,1350,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...
621,Panetone,,,,,,,,,,,,-20
622,Creme de leite,,,,,,,,,,,,"-10,2"
623,Taxi 18/12/2015,,,,,,,,,,,,-25
624,,,,,,,,,,,,,


Com o dataframe preparado podemos realizar a reestruturação dos dados da mesma forma que fizemos em 2014.

In [ ]:
# Criação do contâiner e dicionário para execução do loop
subsets_2015 = []
map_datas2015 = {
    'Janeiro': '01/01/2015',
    'Fevereiro': '01/02/2015',
    'Março': '01/03/2015',
    'Abril': '01/04/2015',
    'Maio': '01/05/2015',
    'Junho': '01/06/2015',
    'Julho': '01/07/2015',
    'Agosto': '01/08/2015',
    'Setembro': '01/09/2015',
    'Outubro': '01/10/2015',
    'Novembro': '01/11/2015',
    'Dezembro': '01/12/2015'
}

for mes , data_despesa in map_datas2015.items():
    # Checar se o mês do dict existe entre as colunas do df
    if mes not in df2015.columns:
        continue

    # Criação do subset com as colunas que contem as descrições e os valores
    subset_df = df2015[['Descrição', mes]].copy()

    # Criamos a coluna de data
    subset_df['Data'] = data_despesa

    # Renomeia a coluna com valores
    subset_df.rename(columns={mes: 'Valor'}, inplace=True)

    # Adiciona o subset ao contâiner
    subsets_2015.append(subset_df)

# Convertemos os subsets do contâiner em um dataframe único
despesas_2015 = pd.concat(subsets_2015, ignore_index=True)

# Removemos linhas sem valor
despesas_2015 = despesas_2015[despesas_2015['Valor'] != '']

# Substituímos o caracter de decimal
despesas_2015['Valor'] = despesas_2015['Valor'].astype(str).str.replace(',', '.').astype(float)

# Definimos o tipo de operação pelo sinal do valor
despesas_2015['Tipo'] = np.where(despesas_2015['Valor'] < 0, 'D', 'R')

# Convertemos de volta para valor para eliminação do sinal
despesas_2015['Valor'] = despesas_2015['Valor'].astype(float).abs()

despesas_2015

,Descrição,Valor,Data,Tipo
0,Salário JV,2746.0,01/01/2015,R
9,Salário Marilia 15,504.0,01/01/2015,R
12,Aluguel,1200.0,01/01/2015,D
17,Taiko,80.0,01/01/2015,D
19,Eletropaulo,51.0,01/01/2015,D
...,...,...,...,...
7505,Mercado,13.0,01/12/2015,D
7506,Presente chefa,20.0,01/12/2015,D
7507,Panetone,20.0,01/12/2015,D
7508,Creme de leite,10.2,01/12/2015,D


Seguindo a mesma lógica de análise de 2014, vamos verificar se temos linhas sem descrição.

In [ ]:
despesas_2015.loc[despesas_2015['Descrição'] == '']

,Descrição,Valor,Data,Tipo


## 2016
Em 2016 o formato da planilha mudou para algo mais próximo do atual, com cada mês utilizando 3 colunas (Descrição, Tipo e Valor).

O tratamento dos dados é um pouco diferente pois não temos mais tantos vazios na planilha mas agora precisamos:
* Isolar as linhas com valores na importação pois temos cabeçalhos neste modelo
* Diferenciar valores de receita e despesa conforme valor (positivo ou negativo)
* Dividir e empilhar a planilha para criar um único df

### Isolar valores

Inicialmente iremos isolar apenas as linhas com valores que serão processados.

In [ ]:
# Importação da aba de 2016
plan2016 = planilha.worksheet('2016')

# Extraímos os dados da planilha já filtrando as colunas
data2016 = plan2016.get_all_records(numericise_ignore=['all'])

# Convertemos para um dataframe para manipulação
df2016 = pd.DataFrame(data2016)

# Vamos filtrar as linhas marcadas como cabeçalho utilizando a primeira
# coluna e então removê-la pois não será mais necessária
df2016 = df2016.loc[df2016['C'] != 'C', :].iloc[:,1:]

df2016

,DESC_JAN,C_JAN,JAN,DESC_FEV,C_FEV,FEV,DESC_MAR,C_MAR,MAR,DESC_ABR,...,SET,DESC_OUT,C_OUT,OUT,DESC_NOV,C_NOV,NOV,DESC_DEZ,C_DEZ,DEZ
10,JV - Quinto dia,0,1560,JV - Quinto dia,0,1560,JV - Quinto dia,0,1560,JV - Quinto dia,...,1461,JV - Quinto dia,0,1565,JV - Quinto dia,0,1455,JV - Quinto dia,0,591
11,JV - Adiantamento,0,"1351,91",JV - Adiantamento,0,"-0,09",JV - Adiantamento,0,"-0,09",JV - Adiantamento,...,,JV - Adiantamento,0,,Férias,0,"4500,72",JV - Adiantamento,0,
12,Ajuste,0,"1881,24",Adiantamento mês seguinte,0,1352,Adiantamento mês seguinte,0,1352,Adiantamento mês seguinte,...,1439,Adiantamento mês seguinte,0,1485,Adiantamento mês seguinte,0,1485,Adiantamento mês seguinte,0,1485
13,13º Marilia,0,,Vale João,0,405,Vale João,0,320,Vale João,...,354,Vale João,0,367,13º João,0,1857,13º João,0,1306
14,Adiantamento mês seguinte,0,1352,Vale Marilia,0,,Vale Marilia,0,522,Vale Marilia,...,0,Mercado,1,0,Mercado,1,,Mercado,NU,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
178,,,,,,,,,,,...,,,,,,,,,,
179,,,,,,,,,,,...,,,,,,,,,,
180,,,,,,,,,,,...,,,,,,,,,,
181,,,,,,,,,,,...,,,,,,,,,,


### Dividir a planilha em subsets

Agora devemos criar um laço parecido com os anteriores, utilizando a coluna principal com valores e com cabeçalho com nome do mês (JAN, FEV, etc) como referência.

In [ ]:
#Criamos a lista e o dict de datas para criar a coluna posteriormente
subsets_2016 = []
map_datas2016 = {
    'JAN': '01/01/2016',
    'FEV': '01/02/2016',
    'MAR': '01/03/2016',
    'ABR': '01/04/2016',
    'MAI': '01/05/2016',
    'JUN': '01/06/2016',
    'JUL': '01/07/2016',
    'AGO': '01/08/2016',
    'SET': '01/09/2016',
    'OUT': '01/10/2016',
    'NOV': '01/11/2016',
    'DEZ': '01/12/2016'
}

#O loop faz a divisão em subsets que são armazenados em uma lista
for mes , data_despesa in map_datas2016.items():
    # Verificamos se o mês do dict existe entre as colunas do df de 2016,
    # caso não exista continuamos para a próxima iteração do loop
    if mes not in df2016.columns:
        continue
    
    #Encontramos o índice da coluna para usar de referência
    i_col = df2016.columns.get_loc(mes)
    
    # Utilizamos o iloc para selecionar as colunas pois estão
    # em posições relativas ao mês
    subset_df = df2016.iloc[:, (i_col - 2):i_col + 1].copy()
    
    #Criamos a coluna de data utilizando o dict criado anteriormente
    subset_df['Data'] = data_despesa
    
    #Renomeamos as colunas para padronizar com os outros anos
    subset_df.rename(
        columns={mes : 'Valor',
                 f'DESC_{mes}':'Descrição',
                 f'C_{mes}':'Código'},
                   inplace=True
                   )
    
    #Armazenamos o subset na lista criada anteriormente
    subsets_2016.append(subset_df)

#Concatenamos a lista com os subsets em um único DataFrame
despesas_2016 = pd.concat(subsets_2016, ignore_index=True)

#Convertemos a coluna de valor para numérico e tratamos os erros
despesas_2016['Valor'] = pd.to_numeric(
    despesas_2016['Valor'].astype(str).str.replace(',', '.'),
    errors='coerce'
)

#Removemos as linhas onde o valor é zero ou nulo
despesas_2016 = despesas_2016.loc[(despesas_2016['Valor'] != 0) & (despesas_2016['Valor'].notna())]

#Criamos a coluna de tipo para diferenciar receitas e despesas
despesas_2016['Tipo'] = np.where(despesas_2016['Código'] == 'R', 'R', 'D')

#Valores positivos com código de despesa 'D' são classificados como 'RE' (Reembolso)
despesas_2016['Tipo'] = np.where((despesas_2016['Tipo'] == 'D') & (despesas_2016['Valor'] > 0), 'RE', despesas_2016['Tipo'])

#Removemos a coluna de código pois não será mais necessária
despesas_2016 = despesas_2016.drop(columns=['Código'])

#Transformamos os valores da coluna valor para positivos
despesas_2016['Valor'] = despesas_2016['Valor'].abs()

despesas_2016

,Descrição,Valor,Data,Tipo
0,JV - Quinto dia,1560.00,01/01/2016,RE
1,JV - Adiantamento,1351.91,01/01/2016,RE
2,Ajuste,1881.24,01/01/2016,RE
4,Adiantamento mês seguinte,1352.00,01/01/2016,RE
5,Vale João,379.00,01/01/2016,RE
...,...,...,...,...
2024,Reembolso de pizza,52.80,01/12/2016,RE
2025,Materiais de Manicure,16.94,01/12/2016,D
2026,Café da manhã,37.50,01/12/2016,D
2027,Reembolso café da manhã,29.20,01/12/2016,RE


In [ ]:
# Verificamos as linhas que possuem descrição vazia
despesas_2016.loc[despesas_2016['Descrição'] == '']

,Descrição,Valor,Data,Tipo


## 2017

Como a planilha manteve sua estrutura em 2017, podemos simplesmente repetir o código, alterando apenas o ano das variáveis.

In [ ]:
#Importamos os dados de 2017 da planilha e criamos o DataFrame
plan2017 = planilha.worksheet('2017')
data2017 = plan2017.get_all_records(numericise_ignore=['all'])
df2017 = pd.DataFrame(data2017)

#Vamos filtrar as linhas marcadas como cabeçalho utilizando a primeira
#coluna e então remover a coluna pois não será mais necessária
df2017 = df2017.loc[df2017['C'] != 'C', :].iloc[:,1:]

#Criamos a lista e o dict de datas para criar a coluna posteriormente
subsets_2017 = []
map_datas2017 = {
    'JAN': '01/01/2017',
    'FEV': '01/02/2017',
    'MAR': '01/03/2017',
    'ABR': '01/04/2017',
    'MAI': '01/05/2017',
    'JUN': '01/06/2017',
    'JUL': '01/07/2017',
    'AGO': '01/08/2017',
    'SET': '01/09/2017',
    'OUT': '01/10/2017',
    'NOV': '01/11/2017',
    'DEZ': '01/12/2017'
}

#O loop faz a divisão em subsets que são armazenados em uma lista
for mes , data_despesa in map_datas2017.items():
    
    # Verificamos se o mês do dict existe entre as colunas do df de 2017
    if mes not in df2017.columns:
        continue
    
    #Encontramos o índice da coluna para usar de referência
    i_col = df2017.columns.get_loc(mes)
    subset_df = df2017.iloc[:, (i_col - 2):i_col + 1].copy()
    
    #Criamos a coluna de data utilizando o dict criado anteriormente
    subset_df['Data'] = data_despesa
    
    #Renomeamos as colunas para padronizar com os outros anos
    subset_df.rename(
        columns={mes : 'Valor',
                 f'DESC_{mes}':'Descrição',
                 f'C_{mes}':'Código'},
                   inplace=True
                   )
    
    #Armazenamos o subset na lista criada anteriormente
    subsets_2017.append(subset_df)

#Concatenamos a lista com os subsets em um único DataFrame
despesas_2017 = pd.concat(subsets_2017, ignore_index=True)

#Convertemos a coluna de valor para numérico e tratamos os erros
despesas_2017['Valor'] = pd.to_numeric(
    despesas_2017['Valor'].astype(str).str.replace(',', '.'),
    errors='coerce'
)

#Removemos as linhas onde o valor é zero ou nulo
despesas_2017 = despesas_2017.loc[(despesas_2017['Valor'] != 0) & (despesas_2017['Valor'].notna())]

#Criamos a coluna de tipo para diferenciar receitas e despesas
despesas_2017['Tipo'] = np.where(despesas_2017['Código'] == 'R', 'R', 'D')

#Valores positivos com código de despesa 'D' são classificados como 'RE' (Reembolso)
despesas_2017['Tipo'] = np.where((despesas_2017['Tipo'] == 'D') & (despesas_2017['Valor'] > 0), 'RE', despesas_2017['Tipo'])

#Removemos a coluna de código pois não será mais necessária
despesas_2017 = despesas_2017.drop(columns=['Código'])

#Transformamos os valores da coluna valor para positivos
despesas_2017['Valor'] = despesas_2017['Valor'].abs()

despesas_2017

,Descrição,Valor,Data,Tipo
0,JV - Quinto dia,582.00,01/01/2017,RE
2,Adiantamento mês seguinte,1485.00,01/01/2017,RE
4,Mercado,28.87,01/01/2017,D
5,Aluguel,1200.00,01/01/2017,D
6,Condomínio,270.39,01/01/2017,D
...,...,...,...,...
1908,Mercadao,5.00,01/12/2017,D
1909,Passagem momoi,20.00,01/12/2017,D
1910,Desenho Luana,65.00,01/12/2017,RE
1911,Encargos,1.13,01/12/2017,D


In [ ]:
despesas_2017.loc[despesas_2017['Descrição'] == '']


,Descrição,Valor,Data,Tipo


## 2018
O processo pode ser repetido novamente em 2018.

In [ ]:
#Importamos os dados de 2018 da planilha e criamos o DataFrame
plan2018 = planilha.worksheet('2018')
data2018 = plan2018.get_all_records(numericise_ignore=['all'])
df2018 = pd.DataFrame(data2018)

#Vamos filtrar as linhas marcadas como cabeçalho utilizando a primeira
#coluna e então remover a coluna pois não será mais necessária
df2018 = df2018.loc[df2018['C'] != 'C', :].iloc[:,1:]

#Criamos a lista e o dict de datas para criar a coluna posteriormente
subsets_2018 = []
map_datas2018 = {
    'JAN': '01/01/2018',
    'FEV': '01/02/2018',
    'MAR': '01/03/2018',
    'ABR': '01/04/2018',
    'MAI': '01/05/2018',
    'JUN': '01/06/2018',
    'JUL': '01/07/2018',
    'AGO': '01/08/2018',
    'SET': '01/09/2018',
    'OUT': '01/10/2018',
    'NOV': '01/11/2018',
    'DEZ': '01/12/2018'
}

#O loop faz a divisão em subsets que são armazenados em uma lista
for mes , data_despesa in map_datas2018.items():
    # Verificamos se o mês do dict existe entre as colunas do df de 2018
    if mes not in df2018.columns:
        continue
    
    #Encontramos o índice da coluna para usar de referência
    i_col = df2018.columns.get_loc(mes)
    subset_df = df2018.iloc[:, (i_col - 2):i_col + 1].copy()
    
    #Criamos a coluna de data utilizando o dict criado anteriormente
    subset_df['Data'] = data_despesa
    
    #Renomeamos as colunas para padronizar com os outros anos
    subset_df.rename(
        columns={mes : 'Valor',
                 f'DESC_{mes}':'Descrição',
                 f'C_{mes}':'Código'},
                  inplace=True
                  )
    
    #Armazenamos o subset na lista criada anteriormente
    subsets_2018.append(subset_df)

#Concatenamos a lista com os subsets em um único DataFrame
despesas_2018 = pd.concat(subsets_2018, ignore_index=True)

#Convertemos a coluna de valor para numérico e tratamos os erros
despesas_2018['Valor'] = pd.to_numeric(
    despesas_2018['Valor'].astype(str).str.replace(',', '.'),
    errors='coerce'
)

#Removemos as linhas onde o valor é zero ou nulo
despesas_2018 = despesas_2018.loc[(despesas_2018['Valor'] != 0) & (despesas_2018['Valor'].notna())]

#Criamos a coluna de tipo para diferenciar receitas e despesas
despesas_2018['Tipo'] = np.where(despesas_2018['Código'] == 'R', 'R', 'D')

#Valores positivos com código de despesa 'D' são classificados como 'RE' (Reembolso)
despesas_2018['Tipo'] = np.where((despesas_2018['Tipo'] == 'D') & (despesas_2018['Valor'] > 0), 'RE', despesas_2018['Tipo'])

#Removemos a coluna de categoria pois não será mais necessária
despesas_2018 = despesas_2018.drop(columns=['Código'])

#Transformamos os valores da coluna valor para positivos
despesas_2018['Valor'] = despesas_2018['Valor'].abs()

despesas_2018

,Descrição,Valor,Data,Tipo
1,JV - Quinto dia,1828.00,01/01/2018,RE
3,Adiantamento mês seguinte,1858.00,01/01/2018,RE
5,Aluguel 7,1200.00,01/01/2018,D
6,Condomínio 15,305.15,01/01/2018,D
7,Taiko 15,100.00,01/01/2018,D
...,...,...,...,...
1686,Emp. Nu B,360.00,01/12/2018,RE
1688,Yuzu Restaurante,110.00,01/12/2018,D
1689,Autopass,10.00,01/12/2018,D
1690,Seguro prestamista,125.22,01/12/2018,RE


In [ ]:
# Verificamos as linhas que possuem descrição vazia
despesas_2018.loc[despesas_2018['Descrição'] == '']


,Descrição,Valor,Data,Tipo


## 2019 a 2025
Em 2019 o processo é semelhante mas vamos adaptar a criação da coluna de tipo de despesa pois agora a maioria dos valores serão positivos mas temos um código específico para valores de receita. Os valores negativos que ainda existe são entradas de reembolso.

Podemos reproduzir este mesmo processo para todos os anos seguintes pois temos o mesmo formato de planilha daqui para frente. Com isso podemos fazer um loop para deixar o código mais resumido.

In [ ]:
# Criamos uma lista com os anos para iterar sobre as abas de 2019 a 2025
abas = [str(ano) for ano in range(2019, 2026)]

# Criamos o contâiner que irá armazenar os subsets
despesas_2019_a_2025 = []

# Criamos um loop acima do loop de subsets para iterar
#  a importação dos dados de cada ano
for ano in abas:

    #Importamos os dados do ano da planilha e criamos o DataFrame
    plan_ano = planilha.worksheet(ano)
    data_ano = plan_ano.get_all_records(numericise_ignore=['all'])
    df_ano = pd.DataFrame(data_ano)
    
    #Vamos filtrar as linhas marcadas como cabeçalho utilizando a primeira
    #coluna e então remover a coluna pois não será mais necessária
    df_ano = df_ano.loc[df_ano['C'] != 'C', :].iloc[:,1:]

    #Criamos a lista e o dict de datas para criar a coluna posteriormente
    subsets_ano = []
    map_datas_ano = {
        'JAN': f'01/01/{ano}',
        'FEV': f'01/02/{ano}',
        'MAR': f'01/03/{ano}',
        'ABR': f'01/04/{ano}',
        'MAI': f'01/05/{ano}',
        'JUN': f'01/06/{ano}',
        'JUL': f'01/07/{ano}',
        'AGO': f'01/08/{ano}',
        'SET': f'01/09/{ano}',
        'OUT': f'01/10/{ano}',
        'NOV': f'01/11/{ano}',
        'DEZ': f'01/12/{ano}'
    }

    #O loop faz a divisão em subsets que são armazenados em uma lista
    for mes , data_despesa in map_datas_ano.items():
        
        # Verificamos se o mês do dict existe entre as colunas do df do ano
        if mes not in df_ano.columns:
            continue
        
        #Encontramos o índice da coluna para usar de referência
        i_col = df_ano.columns.get_loc(mes)
        subset_df = df_ano.iloc[:, (i_col - 2):i_col + 1].copy()
        
        #Criamos a coluna de data utilizando o dict criado anteriormente
        subset_df['Data'] = data_despesa
        
        #Renomeamos as colunas para padronizar com os outros anos
        subset_df.rename(
            columns={mes : 'Valor',
                     f'DESC_{mes}':'Descrição',
                     f'C_{mes}':'Código'},
                       inplace=True
                       )
        
        #Armazenamos o subset na lista criada anteriormente
        subsets_ano.append(subset_df)

    #Concatenamos a lista com os subsets em um único DataFrame
    subset_2019_a_2025 = pd.concat(subsets_ano, ignore_index=True)
    
    #Convertemos a coluna de valor para numérico e tratamos os erros
    subset_2019_a_2025['Valor'] = pd.to_numeric(
        subset_2019_a_2025['Valor'].astype(str).str.replace(',', '.'),
        errors='coerce'
    )
    
    #Adicionamos o subset ao DataFrame principal
    despesas_2019_a_2025.append(subset_2019_a_2025)

#Concatenamos todos os subsets em um único DataFrame
despesas_2019_a_2025 = pd.concat(despesas_2019_a_2025, ignore_index=True)

#Removemos as linhas onde o valor é zero ou nulo
despesas_2019_a_2025 = despesas_2019_a_2025.loc[(despesas_2019_a_2025['Valor'] != 0) & (despesas_2019_a_2025['Valor'].notna())]

#Criamos a coluna de tipo para diferenciar receitas e despesas
despesas_2019_a_2025['Tipo'] = np.where(despesas_2019_a_2025['Código'] == 'R', 'R', 'D')

#Valores positivos com código de despesa 'D' são classificados como 'RE' (Reembolso)
despesas_2019_a_2025['Tipo'] = np.where((despesas_2019_a_2025['Tipo'] == 'D') & (despesas_2019_a_2025['Valor'] < 0), 'RE', despesas_2019_a_2025['Tipo'])

#Removemos a coluna de categoria pois não será mais necessária
despesas_2019_a_2025 = despesas_2019_a_2025.drop(columns=['Código'])

#Transformamos os valores da coluna valor para positivos
despesas_2019_a_2025['Valor'] = despesas_2019_a_2025['Valor'].abs()

despesas_2019_a_2025

,Descrição,Valor,Data,Tipo
0,Salário,3785.0,01/01/2019,R
1,13º,383.0,01/01/2019,R
2,Presente YB,300.0,01/01/2019,R
4,7 - Aluguel,1200.0,01/01/2019,D
5,15 - Condomínio,259.0,01/01/2019,D
...,...,...,...,...
18546,Karen,16.0,01/12/2025,R
18547,Flavia,48.0,01/12/2025,R
18548,Faxina,230.0,01/12/2025,D
18549,Karine,8.0,01/12/2025,R


In [ ]:
# Verificamos as linhas que possuem descrição vazia
despesas_2019_a_2025.loc[despesas_2019_a_2025['Descrição'] == '']


,Descrição,Valor,Data,Tipo


## Categorização das despesas até 2025

O próximo passo será a categorização das despesas em grupos que permitam uma avaliação do perfil de gastos ao longo do tempo.

Para isso vamos unificar os dataframes até o momento e extrair os valores únicos da coluna descrição para criar uma tabela de/para que será exportada para o excel para categorização manual.

In [ ]:
# Concatenanamos todas as listas geradas num único DataFrame
despesas_ant_2025 = pd.concat([despesas_2014,
                               despesas_2015,
                               despesas_2016,
                               despesas_2017,
                               despesas_2018,despesas_2019_a_2025],
                               ignore_index=True)

# Extraímos a lista de descrições únicas para criar a tabela de correspondência
desc_unicas = despesas_ant_2025['Descrição'].loc[despesas_ant_2025['Descrição'] != ''].unique().tolist()

# Transformamos a lista de descrições únicas num DataFrame
desc_unicas = pd.DataFrame(desc_unicas, columns=['Descrição'])

desc_unicas

# A tabela de correspondência é salva em um arquivo Excel para ser preenchida posteriormente
desc_unicas.to_excel('Tabela_De_Para_Para_Preencher.xlsx', index=False)

### Categorias que serão utilizadas
As categorias principais são listadas abaixo:
* Essenciais
    * Aluguel ou prestação do imóvel, condomínio, IPTU, água, luz, gás, internet, telefone.
* Mercado
    * Supermercado, feiras, açougues, padarias, restaurantes, lanches, drogaria.
* Transporte
    * Combustível, estacionamento, transporte público, aplicativos.
* Veículo
    * Parcelas, manutenções, IPVA, Seguros.
* Saúde
    * Plano de saúde, consultas, exames, medicamentos, terapias.
* Educação
    * Mensalidades (escola, cursos), material escolar, livros, uniformes.
* Despesas Pessoais
    * Roupas, calçados, produtos de higiene, cosméticos, academia, cabeleireiro.
* Lazer e Entretenimento
    * Streaming, cinema, shows, passeios, viagens, hobbies.
* Financeiro
    * Parcelas de empréstimos, tarifas, juros, impostos.
* Investimentos e Poupança
    * Aportes em investimentos, reserva de emergência, poupança para objetivos.
* Delivery
    * Jantares fora, pedir comida. Alimentação que não foi preparada em casa.
* Receitas
    * Recebíveis de qualquer espécie
* Casa
    * Gastos com manutenção e aquisições para a casa
* Pets
    * Despesas com animais como ração, plano de saúde e consultas veterinárias
* Outros Gastos
    * Presentes, doações, manutenções inesperadas e despesas eventuais.

## Categorização do histórico

No DataFrame com o histórico até 2025 iremos criar a coluna de categoria utilizando a tabela de-para com as categorias:

In [ ]:
# Obtemos o caminho para a tabela preenchida
de_para_path = os.path.join(script_dir, 'Tabela_De_Para_Preenchida.xlsx')

# Armazenamos o dataframe
de_para_2014_2025 = pd.read_excel(de_para_path)

# Criamos um dicionário a partir da tabela de correspondência
de_para_2014_2025_dict = de_para_2014_2025.set_index('Descrição')['Categoria'].to_dict()

# Criamos a coluna de categoria com base no mapeamento
despesas_ant_2025['Categoria'] = despesas_ant_2025['Descrição'].map(de_para_2014_2025_dict)

despesas_ant_2025


e:\Python\Lib\site-packages\openpyxl\worksheet\_reader.py:329: UserWarning: Data Validation extension is not supported and will be removed
  warn(msg)


,Descrição,Valor,Data,Tipo,Categoria
0,Aluguel,925.0,01/03/2014,D,Essenciais
1,Condomínio,289.0,01/03/2014,D,Essenciais
2,Tanomoshi,3700.0,01/03/2014,R,Financeiro
3,Mobilia,1262.0,01/03/2014,D,Casa
4,Taiko,60.0,01/03/2014,D,Lazer e Entretenimento
...,...,...,...,...,...
17150,Karen,16.0,01/12/2025,R,Financeiro
17151,Flavia,48.0,01/12/2025,R,Financeiro
17152,Faxina,230.0,01/12/2025,D,Essenciais
17153,Karine,8.0,01/12/2025,R,Financeiro


Vamos verificar se temos alguma célula vazia na descrição ou nos valores:

In [ ]:
despesas_ant_2025.loc[(despesas_ant_2025['Descrição'] == '') | (despesas_ant_2025['Valor'] == '')].size

0

## Tratamento dos dados pós 2026
A partir de 2026 alteramos o input de dados na tabela para que o usuário passe a categorizar as entradas junto com sua inserção, deixando o código responsável apenas pela estruturação e disponibilização para consumo pelo pipeline.

O formato da tabela segue o mesmo dos últimos anos, apenas acrescentando uma coluna de categoria em cada mês. Vamos ajustar o procedimento a partir de 2019 para que considere esta coluna

In [ ]:
# Cria uma lista com os nomes das abas a partir de 2026
abas_pos_2026 = [aba.title 
                 for aba in planilha.worksheets(exclude_hidden=True) 
                 if aba.title not in [str(i) for i in range(2014, 2026)]]

# Criamos o contâiner que irá armazenar os subsets
despesas_pos_2026 = []

# Realizamos o loop para importar os dados de cada aba
for ano in abas_pos_2026:

    #Importamos os dados do ano da planilha e criamos o DataFrame
    plan_ano = planilha.worksheet(ano)
    data_ano = plan_ano.get_all_records(numericise_ignore=['all'])
    df_ano = pd.DataFrame(data_ano)
    
    #Vamos filtrar as linhas marcadas como cabeçalho utilizando a primeira
    #coluna e então remover a coluna pois não será mais necessária
    df_ano = df_ano.loc[df_ano['C'] != 'C', :].iloc[:,1:]

    #Criamos a lista e o dict de datas para criar a coluna posteriormente
    subsets_ano = []
    map_datas_ano = {
        'JAN': f'01/01/{ano}',
        'FEV': f'01/02/{ano}',
        'MAR': f'01/03/{ano}',
        'ABR': f'01/04/{ano}',
        'MAI': f'01/05/{ano}',
        'JUN': f'01/06/{ano}',
        'JUL': f'01/07/{ano}',
        'AGO': f'01/08/{ano}',
        'SET': f'01/09/{ano}',
        'OUT': f'01/10/{ano}',
        'NOV': f'01/11/{ano}',
        'DEZ': f'01/12/{ano}'
    }

    #O loop faz a divisão em subsets que são armazenados em uma lista
    for mes , data_despesa in map_datas_ano.items():
        
        # Verificamos se o mês do dict existe entre as colunas do df do ano
        if mes not in df_ano.columns:
            continue
        
        #Encontramos o índice da coluna para usar de referência
        i_col = df_ano.columns.get_loc(mes)
        subset_df = df_ano.iloc[:, (i_col - 3):i_col + 1].copy()
        
        #Criamos a coluna de data utilizando o dict criado anteriormente
        subset_df['Data'] = data_despesa
        
        #Renomeamos as colunas para padronizar com os outros anos
        subset_df.rename(
            columns={mes : 'Valor',
                     f'DESC_{mes}':'Descrição',
                     f'C_{mes}':'Código',
                     f'CAT_{mes}':'Categoria'
                     },
                       inplace=True
                       )
        #Armazenamos o subset na lista criada anteriormente
        subsets_ano.append(subset_df)

    #Concatenamos a lista com os subsets em um único DataFrame
    subset_pos_2026 = pd.concat(subsets_ano, ignore_index=True)
    
    #Convertemos a coluna de valor para numérico e tratamos os erros
    subset_pos_2026['Valor'] = pd.to_numeric(
        subset_pos_2026['Valor'].astype(str).str.replace(',', '.'),
        errors='coerce'
    )
    
    #Adicionamos o subset ao DataFrame principal
    despesas_pos_2026.append(subset_pos_2026)

#Concatenamos todos os subsets em um único DataFrame
despesas_pos_2026 = pd.concat(despesas_pos_2026, ignore_index=True)

#Removemos as linhas onde o valor é zero ou nulo
despesas_pos_2026 = despesas_pos_2026.loc[(despesas_pos_2026['Valor'] != 0) & (despesas_pos_2026['Valor'].notna())]

#Criamos a coluna de tipo para diferenciar receitas e despesas
despesas_pos_2026['Tipo'] = np.where(despesas_pos_2026['Código'] == 'R', 'R', 'D')

#Valores positivos com código de despesa 'D' são classificados como 'RE' (Reembolso)
despesas_pos_2026['Tipo'] = np.where((despesas_pos_2026['Tipo'] == 'D') & (despesas_pos_2026['Valor'] < 0), 'RE', despesas_pos_2026['Tipo'])

#Removemos a coluna de código pois não será mais necessária
despesas_pos_2026 = despesas_pos_2026.drop(columns=['Código'])

#Transformamos os valores da coluna valor para positivos
despesas_pos_2026['Valor'] = despesas_pos_2026['Valor'].abs()

despesas_pos_2026

,Descrição,Categoria,Valor,Data,Tipo
2,04 - Luz,Essenciais,186.24,01/01/2026,D
3,21 - Vivo,Essenciais,100.00,01/01/2026,D
4,Gás,Essenciais,125.00,01/01/2026,D
5,17 - Vivo,Essenciais,150.00,01/01/2026,D
6,Seguro Carro,Veículo,378.60,01/01/2026,D
...,...,...,...,...,...
2014,Combustível,Transporte,289.00,01/12/2026,D
2015,Estac/Ped,Transporte,203.00,01/12/2026,D
2016,iFood,Delivery,931.00,01/12/2026,D
2017,Faxina,Essenciais,460.00,01/12/2026,D


Agora podemos unificar os dataframes `despesas_ant_2025` e `despesas_pos_2026` para criar a base que será consumida pelo Dashboard.

In [ ]:
base_despesas_pessoais = pd.concat([despesas_ant_2025, despesas_pos_2026], ignore_index=True)
base_despesas_pessoais

,Descrição,Valor,Data,Tipo,Categoria
0,Aluguel,925.0,01/03/2014,D,Essenciais
1,Condomínio,289.0,01/03/2014,D,Essenciais
2,Tanomoshi,3700.0,01/03/2014,R,Financeiro
3,Mobilia,1262.0,01/03/2014,D,Casa
4,Taiko,60.0,01/03/2014,D,Lazer e Entretenimento
...,...,...,...,...,...
17781,Combustível,289.0,01/12/2026,D,Transporte
17782,Estac/Ped,203.0,01/12/2026,D,Transporte
17783,iFood,931.0,01/12/2026,D,Delivery
17784,Faxina,460.0,01/12/2026,D,Essenciais
